# Lab: using clustering to find best store locations

Imagine the following situation:<br>
You own a pizza chain, and you collected data about pizza deliveries in a certain neighborhood. The data contains a coordinate of each delivery as a pair *(Latitude, Longitude)*. You do not have any stores in this neighborhood, and driving there each time is too expensive (especially with current gas prices). So you decide to open $K$ new stores in this area. The task is, based on the frequent delivery data, determine the best locations for the new stores.

You need to perform the $K$-means clustering of delivery locations, and output the best location for $K$ new stores. How would you choose the location of the store within each cluster that minimizes the overall distance between the store and each delivery address? __Explain your idea in a separate cell below.__

I would choose a location with the overall minimum Euclidean distance between each delivery address.

The data is 2-dimensional and it is easy to plot it to see if the locations of new stores make sense.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import pandas as pd
import numpy as np

The data is a real data collected by [this person](https://github.com/angelddaz) while they were working as a pizza delivery driver. The file `pizza_delivery_locations.csv` is a projection of the original data and contains only *(Latitude, Longitude)* of each delivery address. 

In [2]:
data_file = "pizza_delivery_locations.csv"

In [39]:
data = pd.read_csv(data_file)
print(data.columns)
len(data)

# convert dataframe to a 2D numpy array - it is easier to work with it
data = data.to_numpy()
print(', '.join(map(str, data[0])))#first row
print(data.shape) #columns, rows
print(data[0][1])

Index(['Latitude', 'Longitude'], dtype='object')
43.666573, -116.263356
(1301, 2)
-116.263356


## Task 1.

Use $K$-means clustering algorithm to find the best locations for new pizza stores for $K$=2, $K$=3 and $K$=4. The answers should be represented as lists of *(Latitide,Longitude)* tuples for each value of $K$.

You can use the custom code from the k-means demo, or you can implement your own clustering algorithm. What distance metric is the most appropriate for this situation?

In [76]:
import numpy as np

def euclidean_distance(x1, y1, x2, y2):
    """
    Calculate the Euclidean distance between two points (x1, y1) and (x2, y2).
    
    Args:
        x1 (float): x-coordinate of the first point.
        y1 (float): y-coordinate of the first point.
        x2 (float): x-coordinate of the second point.
        y2 (float): y-coordinate of the second point.
        
    Returns:
        float: The Euclidean distance between the two points.
    """
    #return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return ((x2 - x1)**2 + (y2 - y1)**2)**0.5

# def kmeans(data, K, max_iters=100):
#     # Randomly initialize K centroids
#     print(data.shape[0])
#     #centroids = data[np.random.choice(data, K, replace=False)]
#     #centroids = data[np.random.choice(data.shape[0], K, replace=False), :]
#     centroids_row_indices = np.random.choice(range(0, data.shape[0]), K, replace=False)
#     print("centroids_row_indices", centroids_row_indices)
#     centroids = []
#     for i in centroids_row_indices:
#         centroids.append(data[i])
    
#     for _ in range(max_iters):
#         # Compute distances between each data point and each centroid
#         #distances = np.array([[euclidean_distance(data_point, centroid) for centroid in centroids] for data_point in data])
#         for i in range(data.shape[0]):
#             for j in range(len(centroids)):
#                 distances = np.array(euclidean_distance(data[i][0], data[i][1], centroids[j][0], centroids[j][1]))
#         print(distances)

        
#         # Assign each data point to the nearest centroid
#         labels = np.argmin(distances, axis=1)
        
#         # Update centroids by taking the mean of all data points assigned to each centroid
#         new_centroids = np.array([data[labels == k].mean(axis=0) for k in range(K)])
        
#         # Check for convergence
#         if np.allclose(centroids, new_centroids):
#             break
        
#         centroids = new_centroids
    
#     return centroids

def kmeans(data, K, max_iters=100):
    centroids_row_indices = np.random.choice(range(0, data.shape[0]), K, replace=False)
    centroids = [data[i] for i in centroids_row_indices]
    
    for _ in range(max_iters):
        distances = []
        for data_point in data:
            distances_to_centroids = [euclidean_distance(data_point[0], data_point[1], centroid[0], centroid[1]) for centroid in centroids]
            distances.append(distances_to_centroids)
        
        distances = np.array(distances)
        
        labels = np.argmin(distances, axis=1)
        
        new_centroids = np.array([data[labels == k].mean(axis=0) for k in range(K)])
        
        if np.allclose(centroids, new_centroids):
            break
        
        centroids = new_centroids
    
    return centroids

kmeans(data, 3)

# def find_best_store_locations(data, K):
#     # Run K-means clustering
#     centroids = kmeans(data, K)
    
#     # Initialize list to store best store locations for each cluster
#     best_store_locations = []
    
#     # Iterate over each centroid
#     for centroid in centroids:
#         # Find the index of the delivery location closest to the centroid
#         closest_index = np.argmin(np.linalg.norm(data - centroid, axis=1))
#         # Get the coordinates of the closest delivery location
#         closest_location = tuple(data[closest_index])
#         # Add the coordinates to the list of best store locations
#         best_store_locations.append(closest_location)
    
#     return best_store_locations

#coordinates = data[:, 1:]
#print("coordinates", coordinates)

# Find the best store locations for K=2, K=3, and K=4
#K_values = [2, 3, 4]
#best_locations_for_K = {K: find_best_store_locations(coordinates, K) for K in K_values}
#print(best_locations_for_K)
# Output the results
#for K, locations in best_locations_for_K.items():
    #print(f"For K={K}, the best store locations are: {locations}")

array([[  43.63263042, -116.20979217],
       [  43.65603272, -116.25422348]])

In [50]:
# Extracting best store locations for each cluster of addresses

# Extract latitude and longitude columns from the data


centroids [[-116.263356]
 [-116.259743]]
centroids [[-116.23413 ]
 [-116.263356]
 [-116.21835 ]]
centroids [[-116.259743]
 [-116.24628 ]
 [-116.286124]
 [-116.23413 ]]
{2: [(-116.259743,), (-116.21835,)], 3: [(-116.24628,), (-116.263356,), (-116.208304,)], 4: [(-116.259743,), (-116.24628,), (-116.286124,), (-116.208304,)]}
For K=2, the best store locations are: [(-116.259743,), (-116.21835,)]
For K=3, the best store locations are: [(-116.24628,), (-116.263356,), (-116.208304,)]
For K=4, the best store locations are: [(-116.259743,), (-116.24628,), (-116.286124,), (-116.208304,)]


## Task 2
Visualize clusters by plotting each data point and coloring it with a different color corresponding to the cluster to which it belongs. Also plot the locations of new stores for each value of $K$. Some examples of the final visualizations are given below.

In [53]:
import matplotlib.pyplot as plt

def plot_clusters(data, centroids, labels, store_locations=None):
    plt.figure(figsize=(8, 6))
    # Plot data points
    for i in range(len(np.unique(labels))):
        cluster_data = data[labels == i]
        print(f"Cluster {i+1} has {len(cluster_data)} data points.")
        print("Data shape:", data.shape)
        print("Cluster data shape:", cluster_data.shape)

        print(cluster_data)
        plt.scatter(cluster_data[:, 0], cluster_data[:, 1], label=f'Cluster {i+1}')
    # Plot centroids
    plt.scatter(centroids[:, 0], centroids[:, 1], marker='X', color='black', label='Centroids')
    # Plot store locations if provided
    if store_locations is not None:
        store_locations = np.array(store_locations)
        plt.scatter(store_locations[:, 0], store_locations[:, 1], marker='o', color='red', label='Store Locations')
    plt.xlabel('Latitude')
    plt.ylabel('Longitude')
    plt.title('K-means Clustering')
    plt.legend()
    plt.grid(True)
    plt.show()

# Visualize clusters and store locations for each value of K
for K, locations in best_locations_for_K.items():
    centroids = kmeans(coordinates, K)
    labels = np.argmin(np.linalg.norm(coordinates[:, None] - centroids, axis=-1), axis=1)
    plot_clusters(coordinates, centroids, labels, store_locations=locations)



1301
centroids [[-116.263356]
 [-116.21835 ]]
Cluster 1 has 705 data points.
Data shape: (1301, 1)
Cluster data shape: (705, 1)
[[-116.263356]
 [-116.259743]
 [-116.263356]
 [-116.263356]
 [-116.24628 ]
 [-116.286124]
 [-116.24628 ]
 [-116.255046]
 [-116.24628 ]
 [-116.24628 ]
 [-116.24628 ]
 [-116.259743]
 [-116.263356]
 [-116.263356]
 [-116.263356]
 [-116.255046]
 [-116.24628 ]
 [-116.24628 ]
 [-116.24628 ]
 [-116.263356]
 [-116.24628 ]
 [-116.24628 ]
 [-116.255046]
 [-116.263356]
 [-116.24628 ]
 [-116.259743]
 [-116.263356]
 [-116.263356]
 [-116.24628 ]
 [-116.263356]
 [-116.259743]
 [-116.286124]
 [-116.259743]
 [-116.263356]
 [-116.259743]
 [-116.255046]
 [-116.255046]
 [-116.263356]
 [-116.263356]
 [-116.24628 ]
 [-116.259743]
 [-116.259743]
 [-116.286124]
 [-116.263356]
 [-116.24628 ]
 [-116.263356]
 [-116.286124]
 [-116.263356]
 [-116.24628 ]
 [-116.286124]
 [-116.24628 ]
 [-116.255046]
 [-116.259743]
 [-116.255046]
 [-116.263356]
 [-116.259743]
 [-116.263356]
 [-116.24628 ]
 [

IndexError: index 1 is out of bounds for axis 1 with size 1

<Figure size 800x600 with 0 Axes>

## Examples of cluster visualization in 2D

Plotting original data:
    
<img src="clusters.png">

Plotting clusters with store locations
<img src="locations.png">

Copyright &copy; 2024 Marina Barsky. All rights reserved.